In [137]:
import os
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import re
import cv2

foreground_threshold = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch

# assign a label to a patch
def patch_to_label(patch):
    df = np.mean(patch)
    if df > foreground_threshold:
        return 1
    else:
        return 0


def mask_to_submission_strings(image_filename):
    """Reads a single image and outputs the strings that should go into the submission file"""
    img_number = int(re.search(r"\d+", image_filename).group(0))
    im = mpimg.imread(image_filename)
    patch_size = 16
    for j in range(0, im.shape[1], patch_size):
        for i in range(0, im.shape[0], patch_size):
            patch = im[i:i + patch_size, j:j + patch_size]
            label = patch_to_label(patch)
            yield("{:03d}_{}_{},{}".format(img_number, j, i, label))


def masks_to_submission(submission_filename, *image_filenames):
    """Converts images into a submission file"""
    with open(submission_filename, 'w') as f:
        f.write('id,prediction\n')
        for fn in image_filenames[0:]:
            f.writelines('{}\n'.format(s) for s in mask_to_submission_strings(fn))


In [138]:
submission_filename = 'dummy_submission.csv'
image_filenames = []
for i in range(1, 51):
    image_filename = 'data/training/groundtruth/satImage_' + '%.3d' % i + '.png'
    print (image_filename)
    image_filenames.append(image_filename)
masks_to_submission(submission_filename, *image_filenames)

data/training/groundtruth/satImage_001.png
data/training/groundtruth/satImage_002.png
data/training/groundtruth/satImage_003.png
data/training/groundtruth/satImage_004.png
data/training/groundtruth/satImage_005.png
data/training/groundtruth/satImage_006.png
data/training/groundtruth/satImage_007.png
data/training/groundtruth/satImage_008.png
data/training/groundtruth/satImage_009.png
data/training/groundtruth/satImage_010.png
data/training/groundtruth/satImage_011.png
data/training/groundtruth/satImage_012.png
data/training/groundtruth/satImage_013.png
data/training/groundtruth/satImage_014.png
data/training/groundtruth/satImage_015.png
data/training/groundtruth/satImage_016.png
data/training/groundtruth/satImage_017.png
data/training/groundtruth/satImage_018.png
data/training/groundtruth/satImage_019.png
data/training/groundtruth/satImage_020.png
data/training/groundtruth/satImage_021.png
data/training/groundtruth/satImage_022.png
data/training/groundtruth/satImage_023.png
data/traini

In [140]:
# load model 
from keras.models import load_model
model = load_model('best_unet_dice_batch1_4depth_16base_400_interpol.h5', compile=False)

In [141]:
# get test images names
test_img_filenames = []
for i in range(1, 51):
    image_filename = 'data/test_set_images/test_' + '%.1d' % i + '/test_'+ '%.1d' % i + '.png'
    test_img_filenames.append(image_filename)
test_img_filenames

['data/test_set_images/test_1/test_1.png',
 'data/test_set_images/test_2/test_2.png',
 'data/test_set_images/test_3/test_3.png',
 'data/test_set_images/test_4/test_4.png',
 'data/test_set_images/test_5/test_5.png',
 'data/test_set_images/test_6/test_6.png',
 'data/test_set_images/test_7/test_7.png',
 'data/test_set_images/test_8/test_8.png',
 'data/test_set_images/test_9/test_9.png',
 'data/test_set_images/test_10/test_10.png',
 'data/test_set_images/test_11/test_11.png',
 'data/test_set_images/test_12/test_12.png',
 'data/test_set_images/test_13/test_13.png',
 'data/test_set_images/test_14/test_14.png',
 'data/test_set_images/test_15/test_15.png',
 'data/test_set_images/test_16/test_16.png',
 'data/test_set_images/test_17/test_17.png',
 'data/test_set_images/test_18/test_18.png',
 'data/test_set_images/test_19/test_19.png',
 'data/test_set_images/test_20/test_20.png',
 'data/test_set_images/test_21/test_21.png',
 'data/test_set_images/test_22/test_22.png',
 'data/test_set_images/test_

In [142]:
#for j in range(0,50):
img = cv2.imread(test_img_filenames[41]) / 255.
img =np.expand_dims(img, 0)
prediction = (model.predict(img)[0,:,:,0] > 0.557).astype(np.uint8)
plt.subplot(121)
plt.imshow(img[0,:,:,:])
plt.subplot(122)
plt.imshow(prediction, cmap='gray')

ValueError: in user code:

    File "/Users/ahmednourachiche/miniforge3/envs/tensorflow/lib/python3.9/site-packages/keras/engine/training.py", line 1621, in predict_function  *
        return step_function(self, iterator)
    File "/Users/ahmednourachiche/miniforge3/envs/tensorflow/lib/python3.9/site-packages/keras/engine/training.py", line 1611, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/ahmednourachiche/miniforge3/envs/tensorflow/lib/python3.9/site-packages/keras/engine/training.py", line 1604, in run_step  **
        outputs = model.predict_step(data)
    File "/Users/ahmednourachiche/miniforge3/envs/tensorflow/lib/python3.9/site-packages/keras/engine/training.py", line 1572, in predict_step
        return self(x, training=False)
    File "/Users/ahmednourachiche/miniforge3/envs/tensorflow/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb

In [134]:
test_imgs = np.array([cv2.imread(test_img_filenames[i]) / 255. for i in range (0,50)])
test_imgs.shape

(50, 608, 608, 3)

In [146]:
import matplotlib.cm as cm
for i in range(1,51):
    img = cv2.resize(test_imgs[i-1],(400,400), interpolation=cv2.INTER_NEAREST) 
    img = np.expand_dims(img, 0)
    prediction = (model.predict(img)[0,:,:,0]).astype(np.uint8)
    filename = 'predictions/test_' + '%.1d' % i+ '.png'
    plt.imsave(filename, prediction.reshape(400,400), cmap=cm.gray)
    

In [145]:
submission_filename = 'trial_2.csv'
image_filenames = []
for i in range(1, 51):
    image_filename = 'predictions/test_' + '%.1d' % i+ '.png'
    print (image_filename)
    image_filenames.append(image_filename)
masks_to_submission(submission_filename, *image_filenames)

predictions/test_1.png
predictions/test_2.png
predictions/test_3.png
predictions/test_4.png
predictions/test_5.png
predictions/test_6.png
predictions/test_7.png
predictions/test_8.png
predictions/test_9.png
predictions/test_10.png
predictions/test_11.png
predictions/test_12.png
predictions/test_13.png
predictions/test_14.png
predictions/test_15.png
predictions/test_16.png
predictions/test_17.png
predictions/test_18.png
predictions/test_19.png
predictions/test_20.png
predictions/test_21.png
predictions/test_22.png
predictions/test_23.png
predictions/test_24.png
predictions/test_25.png
predictions/test_26.png
predictions/test_27.png
predictions/test_28.png
predictions/test_29.png
predictions/test_30.png
predictions/test_31.png
predictions/test_32.png
predictions/test_33.png
predictions/test_34.png
predictions/test_35.png
predictions/test_36.png
predictions/test_37.png
predictions/test_38.png
predictions/test_39.png
predictions/test_40.png
predictions/test_41.png
predictions/test_42.png
p